### What we'll learn:

1. S3 Buckets - Boto3
2. IAM Roles and Users
3. Complete Infrastructure of AWS Sagemaker - Training, Endpoints in SageMaker (No need for deployment in EC2)


In [1]:
import sagemaker
from sklearn.model_selection import train_test_split
import boto3
import pandas as pd

sm_boto3=boto3.client("sagemaker")
sess=sagemaker.Session()
region=sess.boto_session.region_name

bucket="mobilesagemaker89"

print("Using bucket"+ bucket)
print("My region"+region)

/Users/sumi/Desktop/awsSageMaker/sageenv/lib/python3.10/site-packages/pydantic/_internal/_fields.py:172: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[11/21/24 19:11:08] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=78292;file:///Users/sumi/Desktop/awsSageMaker/sageenv/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=149375;file:///Users/sumi/Desktop/awsSageMaker/sageenv/lib/python3.10/site-packages/botocore/credentials.py#1278\1278]8;;\

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/sumi/Library/Application Support/sagemaker/config.yaml


[11/21/24 19:11:09] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=319391;file:///Users/sumi/Desktop/awsSageMaker/sageenv/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=880064;file:///Users/sumi/Desktop/awsSageMaker/sageenv/lib/python3.10/site-packages/botocore/credentials.py#1278\1278]8;;\

Using bucketmobilesagemaker89
My regionus-east-1


In [2]:
df=pd.read_csv("mob_price_classification_train.csv")
df.head()


,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [3]:
df.shape

(2000, 21)

In [4]:
df.isnull().sum()

battery_power    0
blue             0
clock_speed      0
dual_sim         0
fc               0
four_g           0
int_memory       0
m_dep            0
mobile_wt        0
n_cores          0
pc               0
px_height        0
px_width         0
ram              0
sc_h             0
sc_w             0
talk_time        0
three_g          0
touch_screen     0
wifi             0
price_range      0
dtype: int64

In [5]:
df['price_range'].value_counts()

price_range
1    500
2    500
3    500
0    500
Name: count, dtype: int64

In [6]:
df.columns

Index(['battery_power', 'blue', 'clock_speed', 'dual_sim', 'fc', 'four_g',
       'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'pc', 'px_height',
       'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time', 'three_g',
       'touch_screen', 'wifi', 'price_range'],
      dtype='object')

In [7]:
features=list(df.columns)
features

['battery_power',
 'blue',
 'clock_speed',
 'dual_sim',
 'fc',
 'four_g',
 'int_memory',
 'm_dep',
 'mobile_wt',
 'n_cores',
 'pc',
 'px_height',
 'px_width',
 'ram',
 'sc_h',
 'sc_w',
 'talk_time',
 'three_g',
 'touch_screen',
 'wifi',
 'price_range']

In [8]:
X=df.drop('price_range',axis=1)
y=df['price_range']

In [9]:
X.shape

(2000, 20)

In [10]:
y

0       1
1       2
2       2
3       2
4       1
       ..
1995    0
1996    2
1997    3
1998    0
1999    3
Name: price_range, Length: 2000, dtype: int64

In [11]:
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.15, random_state=42)


In [12]:
X_train.shape, X_test.shape

((1700, 20), (300, 20))

In [13]:
y_train.shape, y_test.shape

((1700,), (300,))

In [14]:
# Save in csv or s3 BUCKET
trainX=pd.DataFrame(X_train)
trainX['price_range']=y_train

testX=pd.DataFrame(X_test)
testX['price_range']=y_test

In [15]:
trainX

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
581,1512,1,0.5,0,8,1,18,0.1,88,3,...,1079,1897,3607,12,10,6,1,1,1,3
76,1114,0,2.8,0,4,1,9,0.4,197,3,...,1040,1071,907,10,7,17,1,1,0,0
1916,1176,1,2.1,0,2,1,62,0.5,168,2,...,674,1455,1534,6,2,17,1,0,0,1
1414,1550,1,2.7,0,2,0,32,0.1,126,7,...,1541,1619,1571,12,3,14,0,0,0,2
780,1042,0,2.2,0,15,1,11,0.6,139,5,...,68,1018,2826,18,0,2,1,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1130,1975,1,1.9,1,2,0,31,0.9,151,1,...,775,1607,3022,13,5,19,0,0,1,3
1294,589,1,0.5,0,1,1,59,0.7,146,8,...,759,1858,362,16,10,6,1,1,1,0
860,1829,1,0.5,0,0,1,15,0.4,160,5,...,729,1267,2080,16,11,12,1,0,1,2
1459,1927,0,0.9,1,3,0,11,0.4,190,8,...,491,1506,2916,16,11,18,0,1,1,3


In [16]:
trainX.to_csv("train-V-1.csv", index=False)
testX.to_csv("test-V-1.csv", index=False)

In [17]:
bucket

'mobilesagemaker89'

In [18]:
# Send data to s3. Sagemaker will take the data for training from s3
sk_prefix="sagemaker/mobile_price_classification/sklearncontainer" # here data will be stored in s3
trainpath=sess.upload_data(path='train-V-1.csv',bucket=bucket, key_prefix=sk_prefix) #  path is the Local path

testpath=sess.upload_data(path='test-V-1.csv', bucket=bucket,key_prefix=sk_prefix)

print(trainpath)
print(testpath)


s3://mobilesagemaker89/sagemaker/mobile_price_classification/sklearncontainer/train-V-1.csv
s3://mobilesagemaker89/sagemaker/mobile_price_classification/sklearncontainer/test-V-1.csv


## Script used by AWS SageMaker to train Model

In [ ]:
%%writefile script.py

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score
import sklearn
import joblib
import boto3
import pathlib
from io import StringIO
import argparse
import os
import numpy as np
import pandas as pd


def model_fn(model_dir):
    clf=joblib.load(os.path.join(model_dir,"model.joblib"))
    
if __name__=="__main__":
    print("[Info] Extracting arguments")
    parser=argparse.ArgumentParser()
    
    ## Hyperparameter 
    parser.add_argument("--n_estimators", type=int, default=100)
    parser.add_argument("--random_state",type=int,default=0)
    
    # Data, model and output directories required by sagemaker
    parser.add_argument("--model-dir", type=str, default=os.environ.get("SM_MODEL_DIR"))
    parser.add_argument("--train",type=str,default=os.environ.get("SM_CHANNEL_TRAIN"))
    parser.add_argument("--test",type=str,default=os.environ.get("SM_CHANNEL_TEST"))
    parser.add_argument("--train-file",type=str,default="train-V-1.csv")
    parser.add_argument("--test-file",type=str,default="test-V-1.csv")
    
    args, _=parser.parse_known_args()
    
    print("Sklearn Version: ", sklearn.__version__)
    print("Joblib Version: ", joblib.__version__)
    
    print("[INFO] Reading data")
    print()
    train_df=pd.read_csv(os.path.join(args.train, args.train_file))
    test_df=pd.read_csv(os.path.join(args.test, args.test_file))
    
    features=list(train_df.columns)      #train_df.drop('price_range',axis=1)
    label=features.pop(-1)                #train_df['price_range']
    
    print("Building training and testing datasets")
    print()
    X_train=train_df[features]
    X_test=test_df[features]
    y_train=train_df[label]
    y_test=test_df[label]
    
    print('Column order: ')
    print(features)
    print()
    
    print("Label column is: ",label)
    print()
    
    print("Data Shape: ")
    print()
    print("---- SHAPE OF TRAINING DATA (85%) ----")
    print(X_train.shape)
    print(y_train.shape)
    print()
    print("---- SHAPE OF TESTING DATA (15%) ----")
    print(X_test.shape)
    print(y_test.shape)
    print()
    
    print("Training Random Forest Model ....")
    print()
    model=RandomForestClassifier(n_estimators=args.n_estimators,random_state=args.random_state,
                                 verbose=2, n_jobs=1)
    
    model.fit(X_train,y_train)
    
    print()
    
    model_path=os.path.join(args.model_dir,"model.joblib")
    joblib.dump(model,model_path)
    
    print("Model saved at" + model_path)
    
    y_pred_test=model.predict(X_test)
    test_acc=accuracy_score(y_test,y_pred_test)
    test_rep=classification_report(y_test, y_pred_test)
    
    print()
    print("---- METRICS RESULTS FOR TESTING DATA ----")
    print()
    print("Total Rows are: ",X_test.shape[0])
    print('[TESTING] Model Accuracy is: ', test_acc)
    print('[TESTING] Testing Report: ')
    print(test_rep)
    
    
    
    
    

Overwriting script.py


### AWS SageMaker Entry Point TO Execute the Training script

In [23]:
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION="0.23-1"

sklearn_estimators=SKLearn(
    entry_point="script.py",
    role="arn:aws:iam::147997138201:role/sagemakersumi", # Set Role in AWS it has access to SageMaker Execution
    instance_count=1,
    instance_type="ml.m5.large",
    framework_version=FRAMEWORK_VERSION,
    base_job_name="RF-custom-sklearn",
    hyperparameters={
        "n_estimators":100,
        "random_state":0
    },
    use_spot_instance=True, # It will create the instance, train the model and delete the instance
    #max_wait=7200,
    max_run=3600
)


In [24]:
# Launch the training job, with asynchronous call
sklearn_estimators.fit({"train": trainpath, "test": testpath}, wait=True)


[11/21/24 19:59:37] INFO     Creating training-job with name:                                       ]8;id=720224;file:///Users/sumi/Desktop/awsSageMaker/sageenv/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=905145;file:///Users/sumi/Desktop/awsSageMaker/sageenv/lib/python3.10/site-packages/sagemaker/session.py#1042\1042]8;;\
                             RF-custom-sklearn-2024-11-21-14-29-33-715                                             

2024-11-21 14:29:40 Starting - Starting the training job...
2024-11-21 14:29:56 Starting - Preparing the instances for training...
2024-11-21 14:30:25 Downloading - Downloading input data...
2024-11-21 14:30:55 Downloading - Downloading the training image..2024-11-21 14:31:41,452 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2024-11-21 14:31:41,456 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-11-21 14:31:41,497 sagemaker_sklearn_container.training INFO     Invoking user training script.
2024-11-21 14:31:41,653 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-11-21 14:31:41,665 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-11-21 14:31:41,677 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-11-21 14:31:41,686 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "ad

### To get the model from s3


In [25]:
sklearn_estimators.latest_training_job.wait(logs="None")
artifact=sm_boto3.describe_training_job(
    TrainingJobName=sklearn_estimators.latest_training_job.name
)["ModelArtifacts"]["S3ModelArtifacts"]



2024-11-21 14:32:00 Starting - Preparing the instances for training
2024-11-21 14:32:00 Downloading - Downloading the training image
2024-11-21 14:32:00 Training - Training image download completed. Training in progress.
2024-11-21 14:32:00 Uploading - Uploading generated training model
2024-11-21 14:32:00 Completed - Training job completed


In [26]:
artifact

's3://sagemaker-us-east-1-147997138201/RF-custom-sklearn-2024-11-21-14-29-33-715/output/model.tar.gz'

### Deploy the Model for Endpoint

In [27]:
from sagemaker.sklearn.model import SKLearnModel
from time import gmtime, strftime

model_name="Custom-sklearn-model-"+strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model=SKLearnModel(
    name=model_name,
    model_data=artifact,
    role="arn:aws:iam::147997138201:role/sagemakersumi",
    entry_point="script.py",
    framework_version=FRAMEWORK_VERSION
)

In [28]:
model

## Endpoint Deployment

In [29]:
endpoint_name=model_name="Custom-sklearn-model-"+strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("EndpointName={}".format(endpoint_name))

predictor=model.deploy(
    initial_instance_count=1,
    instance_type="ml.m4.xlarge",
    endpoint_name=endpoint_name
)

EndpointName=Custom-sklearn-model-2024-11-21-14-50-30


[11/21/24 20:20:35] INFO     Creating model with name: Custom-sklearn-model-2024-11-21-14-48-00     ]8;id=543484;file:///Users/sumi/Desktop/awsSageMaker/sageenv/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=31943;file:///Users/sumi/Desktop/awsSageMaker/sageenv/lib/python3.10/site-packages/sagemaker/session.py#4025\4025]8;;\

[11/21/24 20:20:37] INFO     Creating endpoint-config with name                                     ]8;id=272469;file:///Users/sumi/Desktop/awsSageMaker/sageenv/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=312995;file:///Users/sumi/Desktop/awsSageMaker/sageenv/lib/python3.10/site-packages/sagemaker/session.py#5820\5820]8;;\
                             Custom-sklearn-model-2024-11-21-14-50-30                                              

                    INFO     Creating endpoint with name Custom-sklearn-model-2024-11-21-14-50-30   ]8;id=715033;file:///Users/sumi/Desktop/awsSageMaker/sageenv/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=505430;file:///Users/sumi/Desktop/awsSageMaker/sageenv/lib/python3.10/site-packages/sagemaker/session.py#4642\4642]8;;\

------!

In [36]:
X_test[0:2]

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
1860,1646,0,2.5,0,3,1,25,0.6,200,2,5,211,1608,686,8,6,11,1,1,0
353,1182,0,0.5,0,7,1,8,0.5,138,8,16,275,986,2563,19,17,19,1,0,0


In [ ]:
print(predictor.predict(X_test[:2].values.tolist()))


In [ ]:
# Must Do - Must Delete the endpoint
sm_boto3.delete_endpoint(endpoint_name=endpoint_name)
# For training we don't need to do it- it will be automatically opening machine, perform training and deleting it